In [1]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    precision_score, recall_score
)

import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [3]:
df1 = pd.read_csv("C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/data/ordinal_encode/train_processed_data.csv")

df2= pd.read_csv("C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/data/ordinal_encode/test_processed_data.csv")

In [4]:
xtrain=df1.drop(labels=["loan_status"],axis=1)
ytrain=df1[["loan_status"]]

In [5]:
xtest=df2.drop(labels=["loan_status"],axis=1)
ytest=df2[["loan_status"]]

In [6]:
rus = RandomUnderSampler(sampling_strategy=1.0, random_state=42)  
X_res, y_res = rus.fit_resample(xtrain, ytrain)

In [17]:

mlflow.set_tracking_uri("file:///C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/mlruns")
mlflow.set_experiment("loan_acceptance_logreg")  # same as logistic regression


with mlflow.start_run(run_name="NeuralNetwork_run"):


    model = Sequential([
        Input(shape=(xtrain.shape[1],)),
        Dense(64, activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.4),
        Dense(32, activation="relu", kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(16, activation="relu", kernel_regularizer=l2(0.01)),
        Dense(1, activation="sigmoid")
    ])


    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(learning_rate=0.001),
        metrics=["accuracy", "Precision", "Recall", "AUC"]
    )

    
    es = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


    mlflow.log_params({
        "optimizer": "Adam",
        "learning_rate": 0.001,
        "loss": "binary_crossentropy",
        "l2_reg": 0.01,
        "dropout": "0.4/0.3",
        "epochs": 100,
        "patience": 5,
        "batch_size": 32
    })

   
    hist = model.fit(
        xtrain, ytrain,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[es],
        verbose=1
    )

    
    y_prob = model.predict(xtest).ravel()
    y_pred = (y_prob >= 0.5).astype(int)

    acc = accuracy_score(ytest, y_pred)
    prec = precision_score(ytest, y_pred)
    rec = recall_score(ytest, y_pred)
    auc = roc_auc_score(ytest, y_prob)


    mlflow.log_metrics({
        "test_accuracy": acc,
        "test_precision": prec,
        "test_recall": rec,
        "test_auc": auc
    })


    mlflow.keras.log_model(model, artifact_path="model")

    print(f"Model logged successfully in MLflow!")
    print(f"AUC={auc:.3f} | Precision={prec:.3f} | Recall={rec:.3f} | Accuracy={acc:.3f}")


Epoch 1/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - AUC: 0.8281 - Precision: 0.6112 - Recall: 0.2716 - accuracy: 0.8712 - loss: 0.5025 - val_AUC: 0.8742 - val_Precision: 0.8658 - val_Recall: 0.2547 - val_accuracy: 0.8877 - val_loss: 0.3283
Epoch 2/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.8778 - Precision: 0.7167 - Recall: 0.3952 - accuracy: 0.8912 - loss: 0.3091 - val_AUC: 0.8855 - val_Precision: 0.6799 - val_Recall: 0.5473 - val_accuracy: 0.8983 - val_loss: 0.2976
Epoch 3/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.8811 - Precision: 0.7400 - Recall: 0.4393 - accuracy: 0.8978 - loss: 0.2967 - val_AUC: 0.8930 - val_Precision: 0.7916 - val_Recall: 0.4751 - val_accuracy: 0.9070 - val_loss: 0.2837
Epoch 4/100
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.8846 - Precision: 0.7458 - Recall: 0.4615 - accuracy: 0.9005 - loss: 0.2910 - val_AUC: 0.8928 - val_Precision: 0.7391 - val_Recall: 0.5547 - val_accuracy: 0.9082 - val_loss: 0.2844
Epoch 5/100
1173/117

2025/11/12 16:29:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 16:29:11 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/11/12 16:29:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model logged successfully in MLflow!
AUC=0.910 | Precision=0.792 | Recall=0.593 | Accuracy=0.921
